In [15]:
# Prelimenary Imports and ENV variable definitions
import csv
import requests
import os
import yfinance as yf

from dotenv import load_dotenv
from pprint import pprint

load_dotenv()
FILE_PATH = os.environ.get("FILE_PATH") 


CIK_IDENTIFIERS = [
    '0001720792',
    '0001099281',
    '0001079114',
    '0001112520',
    '0001641864',
    '0000846222',
    '0001709323',
    '0000732905',
    '0000883965',
    '0001067983',
    '0001061768',
]

From the SUBMISSION table fetch a list of ACCESSION_NUMBER(s) using the CIK identifiers in table A-1 (Appendix).




In [16]:

picked_submissions = []
with open(FILE_PATH + 'SUBMISSION_2023_q1.tsv', 'r', encoding='utf-8') as q1:
    for submission in csv.DictReader(q1, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])

with open(FILE_PATH + 'SUBMISSION_2023_q2.tsv', 'r', encoding='utf-8') as q2:
    for submission in csv.DictReader(q2, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])

with open(FILE_PATH + 'SUBMISSION_2023_q3.tsv', 'r', encoding='utf-8') as q3:
    for submission in csv.DictReader(q3, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])
    

pprint(picked_submissions)


['0000950123-23-002585',
 '0001172661-23-001336',
 '0001709323-23-000001',
 '0000919574-23-001107',
 '0001567619-23-002543',
 '0001112520-23-000006',
 '0001213900-23-001958',
 '0001099281-23-000009',
 '0001641864-23-000002',
 '0000950123-23-001270',
 '0001821268-23-000059',
 '0001641864-23-000004',
 '0001099281-23-000012',
 '0001709323-23-000004',
 '0001172661-23-002282',
 '0000919574-23-003116',
 '0000950123-23-005270',
 '0000950123-23-004966',
 '0001821268-23-000090',
 '0001567619-23-006061',
 '0001099281-23-000010',
 '0001641864-23-000003',
 '0001112520-23-000008',
 '0001213900-23-026907',
 '0000919574-23-004553',
 '0001172661-23-003174',
 '0001709323-23-000005',
 '0000950123-23-007916',
 '0000950123-23-008074',
 '0001099281-23-000013',
 '0001213900-23-056683',
 '0001112520-23-000013',
 '0001567619-23-006910',
 '0001821268-23-000158',
 '0001641864-23-000005']


From the INFOTABLE fetch a list of NAMEOFISSUER(s) using the ACCESSION_NUMBER(s) created in (b). Use CUSIP(s) to map between brokers since it is unique where names differ slightly.

In [17]:
names_of_issuers = set()

with open(FILE_PATH + 'INFOTABLE_2023_q1.tsv', 'r', encoding='utf-8') as q1:
    for entry in csv.DictReader(q1, delimiter="\t"):
        if entry["ACCESSION_NUMBER"] in picked_submissions:
            names_of_issuers.add(entry["CUSIP"].upper())
            # if entry["CUSIP"] not in names_of_issuers:
            #     names_of_issuers[entry["CUSIP"]] = set()
            # names_of_issuers[entry["CUSIP"]].add(entry["NAMEOFISSUER"])

with open(FILE_PATH + 'INFOTABLE_2023_q2.tsv', 'r', encoding='utf-8') as q2:
    for entry in csv.DictReader(q2, delimiter="\t"):
        if entry["ACCESSION_NUMBER"] in picked_submissions:
            names_of_issuers.add(entry["CUSIP"].upper())
            # if entry["CUSIP"] not in names_of_issuers:
            #     names_of_issuers[entry["CUSIP"]] = set()
            # names_of_issuers[entry["CUSIP"]].add(entry["NAMEOFISSUER"])

with open(FILE_PATH + 'INFOTABLE_2023_q3.tsv', 'r', encoding='utf-8') as q3:
    for entry in csv.DictReader(q3, delimiter="\t"):
        if entry["ACCESSION_NUMBER"] in picked_submissions:
            names_of_issuers.add(entry["CUSIP"].upper())
            # if entry["CUSIP"] not in names_of_issuers:
            #     names_of_issuers[entry["CUSIP"]] = set()
            # names_of_issuers[entry["CUSIP"]].add(entry["NAMEOFISSUER"])
    
pprint(names_of_issuers)


Now we need to convert the CUSIP to tickers, we will do this using the polygon API to fetch info about a holding by it's CUSIP ID. 

Simply download the last 2 most recent file from https://www.sec.gov/data/foiadocsfailsdatahtm and store in dataset folder.

In this step we lose about 12% of the dataset... Unsure if there is a better way to resolve this

In [ ]:


tickers = set()
with open(FILE_PATH + 'cnsfails202311a','r') as f:
    for entry in csv.DictReader(f, delimiter="|"):
        if entry['CUSIP'] in names_of_issuers: 
            tickers.add(entry['SYMBOL'])
            names_of_issuers.remove(entry['CUSIP'])
with open(FILE_PATH + 'cnsfails202310b','r') as f:
    for entry in csv.DictReader(f, delimiter="|"):
        if entry['CUSIP'] in names_of_issuers: 
            tickers.add(entry['SYMBOL'])
            names_of_issuers.remove(entry['CUSIP'])
    
pprint(tickers)
# res = requests.get(URL, params={"cusip":names_of_issuers[0],"active": True, "apiKey": API_KEY})
# ticker = yf.Ticker(names_of_issuers[0])

{'request_id': 'a8cf791ed655913fc7e41f7680b5a2fb',
 'results': [],
 'status': 'OK'}
['G06536125', '531229409', '531229607', '485859110', '31809Y202', '531229854', '64132K102', '87424L108', '090040AF3', 'G6882C106']
set()
